In [1]:
import json
import pandas as pds  
import numpy
import numpy as np
from PIL import Image
import math
import random

from __future__ import print_function, division
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
def export_modified_json_dataset(json_file, output_json, export_json = False, num_files = 500):
    
    filename = open(json_file,)

    annot= json.load(filename)

    #total_images = 1499
    total_images = num_files


    #print(annotations_json.keys())
    #print(len(annot['scene_annotations']))

    #dict_format = {'image_id': 0,  'flickr_url': '', 'filename': '', 'og_width': 0, 'og_height': 0, 'scene_id': [], 'bboxes':[]}

    custom_dataset = []

    for cur_id in range(0, total_images):

        #print(cur_id)

        temp = {'image_id': 0,  'image_url': '', 'filename': '', 'og_width': 0, 'og_height': 0, 'scene_id': [], 'bboxes':[]}

        temp['image_id'] = cur_id

        for element1 in (elementb for elementb in annot["images"] if elementb["id"] == cur_id):

            temp['image_url'] = element1["flickr_url"]
            temp['filename'] = element1['file_name']
            temp['og_width'] = element1['width']
            temp['og_height'] = element1['height']
        
        for element2 in annot["annotations"]:

            if element2["image_id"] == cur_id:
                temp['bboxes'].append(element2["bbox"])

        for element3 in (elementb for elementb in annot['scene_annotations'] if elementb["image_id"] == cur_id):

             settemp= set(element3["background_ids"])
             temp['scene_id'] = list(settemp)
            

        
        #print(temp)
        custom_dataset.append(temp)

    

    scene_background_ids = [{"id": 0, "name": "Clean"}, {"id": 1, "name": "Indoor, Man-made"}, {"id": 2, "name": "Pavement"}, {"id": 3, "name": "Sand, Dirt, Pebbles"}, {"id": 4, "name": "Trash"}, {"id": 5, "name": "Vegetation"}, {"id": 6, "name": "Water"}]


    

    custom_dataset_json = {"data_subset": custom_dataset}


    if (export_json == True):
        fout = open(output_json,)
        json.dump(custom_dataset_json , fout)


    return custom_dataset



In [58]:

def create_custom_boxes(trash_image_annot):

    final_dataset = {'image_ids' : [] , 'boundry_box' : [], 'trash' : [], 'environment' : []}

    for element in trash_image_annot:

        #obtain original dimensions
        width = element['og_width']
        height = element['og_height']
        environment_ids = element['scene_id']

        #      "bbox": [x,y,width,height]
        #add bbox images to data for positive trash
        for box in element['bboxes']:

            x = math.floor(box[0])
            y = math.floor(box[1])
            widthb = box[2]
            heightb = box[3]

            #print(box)

            
            
            #make ROI boxes square
            if (widthb < heightb):
                if (((x + math.ceil(widthb / 2) + (heightb - widthb)) < width)) and (((x - math.ceil(widthb / 2) - (heightb - widthb)) > 0)):
                    widthb = heightb

            if (widthb > heightb):
                if (((y + math.ceil(heightb / 2) + (widthb -heightb )) < height)) and (((y - math.ceil(heightb / 2) - (widthb - heightb)) > 0)):
                    heightb = widthb


            overlaps = False

            dimension_increaser = 0

            while(overlaps == False):
                
                temp = dimension_increaser + 1

                

                if (x + temp + math.ceil(widthb / 2)) > width or (x - temp - math.ceil(widthb / 2)) < 0 or (y - temp - math.ceil(heightb / 2)) < 0 or (y + temp + math.ceil(heightb / 2)) > height:
                    overlaps = True
                    break

                for new_box in element['bboxes']:
                    new_x = new_box[0]
                    new_y = new_box[1]
                    new_widthb = new_box[2]
                    new_heightb = new_box[3]

                    new_x1 = new_box[0] - math.ceil(new_box[2]/2)
                    new_x2 = new_box[0] + math.ceil(new_box[2]/2)
                    new_y1 = new_box[1] - math.ceil(new_box[3]/2)
                    new_y2 = new_box[1] + math.ceil(new_box[3]/2)

                    adj_x2 = (x + temp + math.ceil(widthb / 2))
                    adj_x1 = (x - temp - math.ceil(widthb / 2))
                    adj_y1 = (y - temp - math.ceil(heightb / 2))
                    adj_y2 = (y + temp + math.ceil(heightb / 2))

                    if (new_x != x) and (new_y != y):

                        if ((new_x1 < adj_x1 < new_x2) or (new_x1 < adj_x2 < new_x2)) and ((new_y1 < adj_y1 < new_y2) or (new_y1 < adj_y2 < new_y2)):

                            overlaps = True
                            break

                        if ((adj_x2 - adj_x1) * (adj_y2 - adj_y1)) > ((width*height)/3):
                            overlaps = True
                            break
                
                if overlaps == False:

                    dimension_increaser = temp
            
            final_dataset['boundry_box'].append([x, y, (widthb + dimension_increaser*2), (heightb + dimension_increaser*2)])
            #print([x, y, (widthb + dimension_increaser*2), (heightb + dimension_increaser*2)])

            final_dataset['image_ids'].append(element["image_id"])

            final_dataset['trash'].append(1)

            final_dataset['environment'].append(element['scene_id'])


        #add bbox images for non trash regions

        

        
        goal_height_width = math.floor(min(width, height) / 4)
        
        for box in element['bboxes']:

            counter = 0
            num_pushed = 0
            fail_rate = 0
            box_x = math.floor(box[0])
            box_y = math.floor(box[1])
            widthb = box[2]
            heightb = box[3]
            x1 = box_x - math.floor((box[2])/2)
            x2 = box_x + math.floor((box[2])/2)
            y1 = box_y - math.floor(heightb/2)
            y2 = box_y + math.floor(heightb/2)


            #print(element['image_id'])
            while (counter != 500 and num_pushed != 1 and fail_rate != 1000):
                
                fail_rate = fail_rate + 1
                #print(fail_rate)
                adj_x = random.randint(0, goal_height_width)
                adj_y = random.randint(0, goal_height_width)
                adj_x1 = adj_x - math.floor(goal_height_width/2)
                adj_x2 = adj_x + math.floor(goal_height_width/2)
                adj_y1 = adj_y - math.floor(goal_height_width/2)
                adj_y2 = adj_y + math.floor(goal_height_width/2)

                if not(((x1 < adj_x1 < x2) or (x1 < adj_x2 < x2)) and ((y1 < adj_y1 < y2) or (y1 < adj_y2 < y2))):

                    
                    counter = counter + 1

                    success = True

                    for obox in element['bboxes']:
                        
                        obox_x = math.floor(obox[0])
                        obox_y = math.floor(obox[1])
                        owidthb = box[2]
                        oheightb = box[3]
                        ox1 = obox_x - math.floor((obox[2])/2)
                        ox2 = obox_x + math.floor((obox[2])/2)
                        oy1 = obox_y - math.floor(oheightb/2)
                        oy2 = obox_y + math.floor(oheightb/2)

                        if (obox_x)!= box_x and (obox_y!= box_y):
                            if (((ox1 < adj_x1 < ox2) or (ox1 < adj_x2 < ox2)) and ((oy1 < adj_y1 < oy2) or (oy1 < adj_y2 < oy2))):
                                success = False

                    if success == True:
                        num_pushed = num_pushed + 1

                        final_dataset['boundry_box'].append([adj_x, adj_y, (goal_height_width), (goal_height_width)])
                        #print([x, y, (widthb + dimension_increaser*2), (heightb + dimension_increaser*2)])

                        final_dataset['image_ids'].append(element["image_id"])

                        final_dataset['trash'].append(0)

                        final_dataset['environment'].append(element['scene_id'])
        

    #print(len(final_dataset['image_ids']))

    
    return final_dataset

In [3]:
def export_roi_dataset(final_dataset, trash_annotations, output_folder):

    images_pil = []

    for i in range(0, len(final_dataset['image_ids'])) :

        image_id = final_dataset['image_ids'][i] 

        filenamey = trash_annotations[ image_id ]['filename']

        pil_image = Image.open ("./image_data/" +  filenamey)

        #full_image = numpy.array (  pil_image  )

        x1 = final_dataset['boundry_box'][i][0] - math.floor(final_dataset['boundry_box'][i][2]/2)
        x2 = final_dataset['boundry_box'][i][0] + math.floor(final_dataset['boundry_box'][i][2]/2)
        y1 = final_dataset['boundry_box'][i][1] - math.floor(final_dataset['boundry_box'][i][3]/2)
        y2 = final_dataset['boundry_box'][i][1] + math.floor(final_dataset['boundry_box'][i][3]/2)



        if (x1 < 0):
            x1 = 0

        if (y1 < 0):
            y1 = 0


        #if (x2 > ((np.shape(full_image))[0]) - 2):
           # x2 = ((np.shape(full_image))[0]) - 2

        #if (y2 > ((np.shape(full_image))[1]) - 2):
            #y2 = ((np.shape(full_image))[0]) - 2

        if x1 == x2:
            x2 = x1 + 1
        
        if y1 == y2:
            y2 = y1 + 1

        
        #new_im = Image.fromarray(full_image[x1:x2, y1:y2, :])


        
        width, height = pil_image.size

        #print(pil_image.size)
        #print(x1, y2, x2, y1)

        #images_np.append(full_image[x1:x2, y1:y2, :])

        new_im = pil_image.crop((x1, height - y2, x2, height - y1))

        new_im = new_im.resize((1000,1000))

        images_pil.append(new_im)

        environments = [str(x) for x in final_dataset['environment'][i]]

        environments = "".join(environments)

        new_im.save(output_folder + str(i) + "_" + str(image_id) + "_" + str(final_dataset['trash'][i]) + "_" + environments + ".png")

    
    #arr_reshaped = images_np(arr.shape[0], -1)
    

    #np.savetxt("./data/numpy_image_data.txt", arr_reshaped)




    # return image_ids, image_data, trash_labels, environment_labels
    return final_dataset["image_ids"], images_pil, final_dataset["trash"], final_dataset["environment"]


In [60]:
    json_file = "./data/annotations.json"

    output_json = "./data/custom_498_data.json"

    trash_image_annot = export_modified_json_dataset(json_file, output_json = False, num_files = 200)

    print(trash_image_annot[156])
    print(len(trash_image_annot))

{'image_id': 156, 'image_url': 'https://farm66.staticflickr.com/65535/48693783818_0e191e045d_o.png', 'filename': 'batch_10/000055.jpg', 'og_width': 4000, 'og_height': 1824, 'scene_id': [3], 'bboxes': [[1323.9048, 959.1428, 296.0, 316.0000000000001]]}
200


In [61]:
final_dataset = create_custom_boxes(trash_image_annot)
print(len(final_dataset['image_ids']))

print(type(final_dataset['environment'][3]))


1309
<class 'list'>


In [62]:
#print(trash_image_annot[223])

image_ids, image_data, trash_labels, environment_labels = export_roi_dataset(final_dataset, trash_image_annot, output_folder = "./new_data/")


    

4000, 1824)
0 507 452 51
(4000, 1824)
0 516 306 60
(4000, 1824)
0 360 232 0
(4000, 1824)
203 551 659 95
(4000, 1824)
0 267 250 0
(4000, 1824)
0 566 372 110
(4000, 1824)
0 549 228 93
(4000, 1824)
37 581 493 125
(4000, 1824)
187 415 643 0
(4000, 1824)
44 460 500 4
(4000, 1824)
0 352 285 0
(4000, 1824)
0 503 246 47
(4000, 1824)
226 469 682 13
(4000, 1824)
0 548 315 92
(1824, 4000)
383 2383 715 2051
(1824, 4000)
629 2631 1289 1971
(1824, 4000)
1197 2737 1471 2463
(1824, 4000)
617 4000 1345 3272
(1824, 4000)
1558 3482 1756 3284
(1824, 4000)
245 3540 1143 2642
(1824, 4000)
1442 2592 1736 2298
(1824, 4000)
1732 3518 1824 3320
(1824, 4000)
1219 2406 1547 2078
(1824, 4000)
1036 2166 1652 1550
(1824, 4000)
1521 2186 1789 1918
(1824, 4000)
0 472 278 16
(1824, 4000)
105 561 561 105
(1824, 4000)
0 311 301 0
(1824, 4000)
0 630 366 174
(1824, 4000)
210 425 666 0
(1824, 4000)
116 494 572 38
(1824, 4000)
0 446 370 0
(1824, 4000)
0 310 421 0
(1824, 4000)
140 329 596 0
(1824, 4000)
210 375 666 0
(1824, 4

NameError: name 'arr' is not defined

In [14]:
print(len(image_data))
print(len(image_ids))
print(len(trash_labels))
print(sum(trash_labels))
print(len(environment_labels))


1933
1933
1933
685
1933


In [13]:
class CustomTrashDataset(Dataset):
    """custom Trash Taco dataset for 498."""

    def __init__(self, image_data, image_ids, trash_labels, environment_labels):
        """
        Args:
            #all lists -> image_data is list of numpy_arrays images 3d
        """
        self.image_data = image_data
        self.image_ids = image_ids
        self.trash_labels = trash_labels
        self.environment_labels = environment_labels

    def __len__(self):
        return len(self.trash_labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()


        get_im_id = self.image_ids[idx]
        get_im_data = self.image_data[idx]
        get_trash_label = self.trash_labels
        get_environment_label = self.environment_labels

        
        sample = {'image_id': get_im_id, 'image_data': get_im_data, 'trash_label' : get_trash_label, 'environment_label' : get_environment_label}
        return sample

        

NameError: name 'Dataset' is not defined

In [10]:
def main():

    
    json_file = "./data/annotations.json"
    

    output_json = "./data/custom_498_data.json"


    trash_image_annot = export_modified_json_dataset(json_file, output_json = False, num_files = 200)


    final_dataset = create_custom_boxes(trash_image_annot)

    

    #all lists -> image_data is list of tensor arrays
    image_ids, image_data, trash_labels, environment_labels = export_roi_dataset(final_dataset, trash_image_annot)

    


if __name__ == "__main__":
    main()


MemoryError: Unable to allocate 22.9 MiB for an array with shape (2448, 3264, 3) and data type uint8